# Import libraries and define directory
Loading necessary libraries and defininf the data file directory

In [1]:
# Import Libraries
from pathlib import Path
import os
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objs as go
import math
import plotly.offline as pyo

# Set directory to where the data file is located
files_directory = Path().resolve()
os.chdir(files_directory)


## Load data
Load the data from the same directory of this code and inspect.

In [3]:
# Load data file
data_filename = input("Enter the name of the data file in the directory where this file is (e.g., 'data.xlsx'): ")
data = pd.read_excel(data_filename)
print(data.head())  # Display first few rows of data

Enter the name of the data file in the directory where this file is (e.g., 'data.xlsx'):  data.xlsx


         ID      HFP      HFM  HFL           X            Y  PROF
0  P1A1-0.2     0.00    0.000  0.0  387798.385  1996486.654   0.2
1  P2A1-0.2   539.26   81.314  0.0  387783.425  1996507.111   0.2
2  P3A1-0.2  1835.69  144.040  0.0  387770.806  1996527.589   0.2
3  P4A1-0.2  2654.56    0.000  0.0  387749.637  1996514.566   0.2
4  P5A1-0.2  2015.54  164.990  0.0  387762.466  1996493.378   0.2


# Define relevant variables
Assign the name of columns for UTM X & Y coordinates, depth, pollutant concentration and its threshold value.

In [7]:
# Define key variables
x_column = input("Enter the name of the column for the X coordinate (e.g., X): ")
y_column = input("Enter the name of the column for the Y coordinate (e.g., Y): ")
z_column = input("Enter the name of the column for depth (e.g., PROF): ")
pollutant_column = input("Enter the name of the column for pollutant concentration (e.g., HFP): ")
MPL = float(input("Enter the Maximum Permissible Limit (MPL) for pollutant concentration (e.g., 3000): "))
units = input("Enter the units for pollutant concentration (e.g., 'mg/kg', 'ppm'): ")

x = data[x_column].values  # UTM X coordinate
y = data[y_column].values  # UTM Y coordinate
z = data[z_column].values * -1  # Depth values (inverted)
pollutant_concentration = data[pollutant_column].values  # Pollutant concentration

Enter the name of the column for the X coordinate (e.g., X):  X
Enter the name of the column for the Y coordinate (e.g., Y):  Y
Enter the name of the column for depth (e.g., PROF):  PROF
Enter the name of the column for pollutant concentration (e.g., HFP):  HFP
Enter the Maximum Permissible Limit (MPL) for pollutant concentration (e.g., 3000):  3000
Enter the units for pollutant concentration (e.g., 'mg/kg', 'ppm'):  'mg/kg'


# Generate 3D grid
Generate a 3D grid to interpolate the data over the defined spatial region.

In [8]:
# Create a 3D grid
grid_x, grid_y, grid_z = np.mgrid[
    min(x):max(x):1,
    min(y):max(y):1,
    min(z):0:0.1
]

## Interpolate data on the 3D grid
Interpolate the pollutant data onto the 3D grid.

In [9]:
# Interpolation
grid_pollutant = griddata((x, y, z), pollutant_concentration, (grid_x, grid_y, grid_z), method='linear')

# 3D visualization of pollutant distribution
Create a 3D visualization of the pollutant distribution using Plotly.

In [ ]:
# 3D Visualization
max_plot = math.ceil(pollutant_concentration.max() / MPL) * MPL
fig = go.Figure(data=go.Volume(
    x=grid_x.flatten(),
    y=grid_y.flatten(),
    z=grid_z.flatten(),
    value=grid_pollutant.flatten(),
    isomin=0,
    isomax=max_plot,
    opacity=0.5,
    surface_count=int(max_plot / MPL)
))

# Set aspect ratio
x_range = max(x) - min(x)
y_range = max(y) - min(y)
ratio_y = y_range / x_range

x_title = input("Enter the X-axis title (e.g., 'UTM E (m)'): ")
y_title = input("Enter the Y-axis title (e.g., 'UTM N (m)'): ")
z_title = input("Enter the X-axis title (e.g., 'Depth (m)'): ")
color_title = input("Enter the color bar title (e.g., 'Concentration'): ")
title = input("Enter the plot title (e.g., 'HFP in soil'): ")

# Configure figure layout
fig.update_layout(
    scene=dict(
        xaxis_title=x_title,
        yaxis_title=y_title,
        zaxis_title=z_title,
        coloraxis_colorbar=dict(
            title=f"{color_title} ({units})"
    ),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=ratio_y, z=0.25),
    title='Pollutant concentration in soil'
)
fig.show()

# Estimation of polluted volume
Estimate the volume where the pollutant concentration exceeds the Maximum Permissible Limit (MPL)

In [10]:
# Create a mask for cells with concentrations above MPL
mask = grid_pollutant > MPL

# Determine grid resolution
delta_x = (max(x) - min(x)) / grid_x.shape[0]
delta_y = (max(y) - min(y)) / grid_y.shape[1]
delta_z = (max(z) - min(z)) / grid_z.shape[2]

# Calculate cell volume
volume_cell = delta_x * delta_y * delta_z

# Count cells exceeding MPL and calculate total polluted volume
num_cells_above = np.sum(mask)
volume_total_above = num_cells_above * volume_cell

print(f"The soil volume with pollutant concentrations above MPL is: {volume_total_above:.2f} m^3")

The soil volume with pollutant concentrations above MPL is: 1122.72 m^3


# Determine maximum contamination depth
Identify the maximum depth where pollutant concentration exceeds MPL.

In [11]:
# Maximum contamination depth
depths_above = grid_z[mask]
maximum_depth = np.min(depths_above)

print(f"Maximum depth with pollutant concentration above MPL: {maximum_depth:.2f} meters")

Maximum depth with pollutant concentration above MPL: -1.20 meters


# 3D visualization of polluted volume
Create a filtered 3D visualization showing only the volume exceeding MPL.

In [ ]:
# Visualization of polluted volume
filtered_grid_x = grid_x[mask]
filtered_grid_y = grid_y[mask]
filtered_grid_z = grid_z[mask]
filtered_pollutant = grid_pollutant[mask]

fig_contaminated = go.Figure(data=go.Volume(
    x=filtered_grid_x.flatten(),
    y=filtered_grid_y.flatten(),
    z=filtered_grid_z.flatten(),
    value=filtered_pollutant.flatten(),
    isomin=MPL,
    isomax=max_plot,
    opacity=0.5,
    surface_count=int((max_plot - MPL) / MPL)
))

fig_contaminated.update_layout(
    scene=dict(
        xaxis_title='UTM X (m)',
        yaxis_title='UTM Y (m)',
        zaxis_title='Depth (m)'
    ),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=ratio_y, z=0.25),
    title=f'Polluted volume (pollutant concentrations > {MPL})'
)
fig_contaminated.show()

# Export visualization as HTML
Save the visualizations as HTML files.

In [ ]:
# Save visualizations as HTML
pyo.plot(fig, filename='volumetric_visualization.html', auto_open=False)
pyo.plot(fig_contaminated, filename='polluted_visualization.html', auto_open=False)